## Mango Db with Python- Part -1

In [ ]:
import pymongo

client=pymongo.MongoClient('mongodb://127.0.0.1:27017/')

mydb=client['Employee']
empinfo=mydb.employeeinformation

In [ ]:
record={
        'firstname':'Krish',
        'lastname':'Naik',
        'department':'Analytics',
        'qualification':'BE',
        'age':29
        }
empinfo.insert_one(record)

In [ ]:
records=[{
        'firstname':'John',
        'lastname':'Doe',
        'department':'Analytics',
        'qualification':'statistics',
        'age':35

        },
         {
        'firstname':'John ',
        'lastname':'Smith',
        'department':'Analytics',
        'qualification':'masters',
        'age':30

        },
        {
        'firstname':'Manish',
        'lastname':'Sen',
        'department':'Analytics',
        'qualification':'phd',
        'age':34

        },
        {
        'firstname':'Ram',
        'lastname':'Singh',
        'department':'Analytics',
        'qualification':'master',
        'age':32

        }]

In [ ]:
empinfo.insert_many(records)

In [ ]:
## Simple way of querying
empinfo.find_one()

{'_id': ObjectId('5e59ec2be0e386955e9489ba'),
 'firstname': 'Krish',
 'lastname': 'Naik',
 'department': 'Analytics',
 'qualification': 'BE',
 'age': 29}

In [ ]:
## Select * from employeeinformation
for record in empinfo.find({}):
    print(record)

{'_id': ObjectId('5e59ec2be0e386955e9489ba'), 'firstname': 'Krish', 'lastname': 'Naik', 'department': 'Analytics', 'qualification': 'BE', 'age': 29}
{'_id': ObjectId('5e59eca7e0e386955e9489bb'), 'firstname': 'John', 'lastname': 'Doe', 'department': 'Analytics', 'qualification': 'statistics', 'age': 35}
{'_id': ObjectId('5e59eca7e0e386955e9489bc'), 'firstname': 'John ', 'lastname': 'Smith', 'department': 'Analytics', 'qualification': 'masters', 'age': 30}
{'_id': ObjectId('5e59eca7e0e386955e9489bd'), 'firstname': 'Manish', 'lastname': 'Sen', 'department': 'Analytics', 'qualification': 'phd', 'age': 34}
{'_id': ObjectId('5e59eca7e0e386955e9489be'), 'firstname': 'Ram', 'lastname': 'Singh', 'department': 'Analytics', 'qualification': 'master', 'age': 32}


In [ ]:
## Query the json documents based on equality conditions
# Select * from employeeinformation where firstname=krish

for record in empinfo.find({'firstname':'Krish'}):
    print(record)


{'_id': ObjectId('5e59ec2be0e386955e9489ba'), 'firstname': 'Krish', 'lastname': 'Naik', 'department': 'Analytics', 'qualification': 'BE', 'age': 29}


In [ ]:
## Query documents using query operators($in,$lt,$gt)
for record in empinfo.find({'qualification':{'$in':['phd','master']}}):
    print(record)

{'_id': ObjectId('5e59eca7e0e386955e9489bd'), 'firstname': 'Manish', 'lastname': 'Sen', 'department': 'Analytics', 'qualification': 'phd', 'age': 34}
{'_id': ObjectId('5e59eca7e0e386955e9489be'), 'firstname': 'Ram', 'lastname': 'Singh', 'department': 'Analytics', 'qualification': 'master', 'age': 32}


In [ ]:
## And and Query operators
for record in empinfo.find({'qualification':'master','age':{'$lt':35}}):
    print(record)

{'_id': ObjectId('5e59eca7e0e386955e9489be'), 'firstname': 'Ram', 'lastname': 'Singh', 'department': 'Analytics', 'qualification': 'master', 'age': 32}


In [ ]:
## OR operators
for record in empinfo.find({'$or':[{'firstname':'Krish'},{'qualification':'BE'}]}):
    print(record)

{'_id': ObjectId('5e59ec2be0e386955e9489ba'), 'firstname': 'Krish', 'lastname': 'Naik', 'department': 'Analytics', 'qualification': 'BE', 'age': 29}


In [ ]:
inventory=mydb.inventory

In [ ]:
inventory.insert_many( [
   { 'item': "journal", 'qty': 25, 'size': { 'h': 14, 'w': 21,'uom': "cm" }, 'status': "A" },
   { 'item': "notebook", 'qty': 50,'size': { 'h': 8.5, 'w': 11,'uom': "in" },'status': "A" },
   { 'item': "paper", 'qty': 100, 'size': { 'h': 8.5, 'w': 11,'uom': "in" },'status': "D" },
   { 'item': "planner", 'qty': 75, 'size': { 'h': 22.85,'w': 30,'uom': "cm" },'status': "D" },
   { 'item': "postcard", 'qty': 45, 'size': { 'h': 10, 'w': 15.25,'uom': "cm" },'status': "A" }
]);


In [ ]:
for records in inventory.find({'size': { 'h': 14, 'w': 21,'uom': "cm" }}):
    print(records)

{'_id': ObjectId('5e59f14ee0e386955e9489bf'), 'item': 'journal', 'qty': 25, 'size': {'h': 14, 'w': 21, 'uom': 'cm'}, 'status': 'A'}


## Mango Db with Python- Part -2

In [ ]:
# Count employees in each department
pipeline = [
    {"$group": {"_id": "$department", "total_employees": {"$sum": 1}}}
]

department_distribution = empinfo.aggregate(pipeline)

# Print the results
for department in department_distribution:
    print(f"Department: {department['_id']}, Total Employees: {department['total_employees']}")


In [ ]:
# Calculate average, min, and max age
pipeline = [
    {"$group": {"_id": None, "average_age": {"$avg": "$age"}, "min_age": {"$min": "$age"}, "max_age": {"$max": "$age"}}}
]

age_stats = empinfo.aggregate(pipeline)

# Print the results
for record in age_stats:
    print(f"Average Age: {record['average_age']:.2f}, Min Age: {record['min_age']}, Max Age: {record['max_age']}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get age data for all employees
ages = [record['age'] for record in empinfo.find({}, {"age": 1, "_id": 0})]

# Plot a histogram of employee ages
sns.histplot(ages, kde=True)
plt.title("Age Distribution of Employees")
plt.xlabel("Age")
plt.ylabel("Number of Employees")
plt.show()


In [ ]:
# Analyze hiring trends by year
pipeline = [
    {"$project": {"year_hired": {"$year": "$hire_date"}}},  # Extract year from hire_date
    {"$group": {"_id": "$year_hired", "total_hired": {"$sum": 1}}}
]

hiring_trends = empinfo.aggregate(pipeline)

# Print results
for record in hiring_trends:
    print(f"Year: {record['_id']}, Employees Hired: {record['total_hired']}")


In [ ]:
# Prepare data
departments = []
employee_count = []

pipeline = [
    {"$group": {"_id": "$department", "total_employees": {"$sum": 1}}}
]
department_distribution = empinfo.aggregate(pipeline)

for department in department_distribution:
    departments.append(department['_id'])
    employee_count.append(department['total_employees'])

# Plot bar chart
sns.barplot(x=departments, y=employee_count)
plt.title("Employee Distribution by Department")
plt.xlabel("Department")
plt.ylabel("Number of Employees")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Prepare data
years = []
hired_count = []

pipeline = [
    {"$project": {"year_hired": {"$year": "$hire_date"}}},
    {"$group": {"_id": "$year_hired", "total_hired": {"$sum": 1}}},
    {"$sort": {"_id": 1}}
]
hiring_trends = empinfo.aggregate(pipeline)

for record in hiring_trends:
    years.append(record['_id'])
    hired_count.append(record['total_hired'])

# Plot line chart
plt.plot(years, hired_count, marker='o')
plt.title("Hiring Trends Over the Years")
plt.xlabel("Year")
plt.ylabel("Number of Employees Hired")
plt.grid(True)
plt.show()
